In [ ]:
import sys
sys.path.append("../../")

In [ ]:
import libcst as cst

source = """
import a, b, c as d, e as f  # expect to keep: a, c as d
from g import h, i, j as k, l as m  # expect to keep: h, j as k
from m import n  # expect to be removed entirely

a()

def fun():
    d()

class Cls:
    att = h.something
    
    def __init__(self) -> None:
        var = k.method()
"""
wrapper = cst.metadata.MetadataWrapper(cst.parse_module(source))
scopes = set(wrapper.resolve(cst.metadata.ScopeProvider).values())
for scope in scopes:
    print(scope)

In [ ]:
from collections import defaultdict
from typing import Dict, Union, Set

import_names_to_remove: Dict[Union[cst.Import, cst.ImportFrom], Set[str]] = defaultdict(set)
for scope in scopes:
    for assignment in scope:
        if isinstance(assignment, cst.metadata.Assignment) and isinstance(
            assignment.node, (cst.Import, cst.ImportFrom)
        ):
            if len(assignment.accesses) == 0:
                import_names_to_remove[assignment.node].add(assignment.name)


In [ ]:
class RemoveUnusedImportTransformer(cst.CSTTransformer):
    def __init__(
        self, import_names_to_remove: Dict[Union[cst.Import, cst.ImportFrom], Set[str]]
    ) -> None:
        self.import_names_to_remove = import_names_to_remove

    def leave_import_alike(
        self,
        original_node: Union[cst.Import, cst.ImportFrom],
        updated_node: Union[cst.Import, cst.ImportFrom],
    ) -> Union[cst.Import, cst.ImportFrom, cst.RemovalSentinel]:
        if original_node not in self.import_names_to_remove:
            return updated_node
        names_to_keep = []
        for name in updated_node.names:
            asname = name.asname
            if asname is not None:
                name_value = asname.name.value
            else:
                name_value = name.name.value
            if name_value not in self.import_names_to_remove[original_node]:
                names_to_keep.append(name.with_changes(comma=cst.MaybeSentinel.DEFAULT))
        if len(names_to_keep) == 0:
            return cst.RemoveFromParent()
        else:
            return updated_node.with_changes(names=names_to_keep)

    def leave_Import(
        self, original_node: cst.Import, updated_node: cst.Import
    ) -> cst.Import:
        return self.leave_import_alike(original_node, updated_node)

    def leave_ImportFrom(
        self, original_node: cst.ImportFrom, updated_node: cst.ImportFrom
    ) -> cst.ImportFrom:
        return self.leave_import_alike(original_node, updated_node)


In [ ]:
fixed_module = wrapper.module.visit(RemoveUnusedImportTransformer(import_names_to_remove))
print(fixed_module.code)